# Gallery Example: M/M/1 Reentrant Network

This example demonstrates an M/M/1 reentrant network where jobs:
- Visit the same station multiple times
- Follow a specific reentrant routing pattern
- Create complex dependencies and potential bottlenecks

Reentrant networks are common in manufacturing and computer systems where jobs require multiple passes through the same resources.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_reentrant():    """Create M/M/1 reentrant network"""    model = Network('M/M/1-Reentrant')        # Block 1: nodes    source = Source(model, 'Source')    queue1 = Queue(model, 'Queue1', SchedStrategy.FCFS)    queue2 = Queue(model, 'Queue2', SchedStrategy.FCFS)     sink = Sink(model, 'Sink')        # Block 2: classes - multiple classes for different routing stages    class1 = OpenClass(model, 'Stage1')  # First visit to Queue1    class2 = OpenClass(model, 'Stage2')  # Visit to Queue2    class3 = OpenClass(model, 'Stage3')  # Second visit to Queue1 (reentrant)        # Arrivals only to first class    source.set_arrival(class1, Exp(0.5))  # External arrivals    source.set_arrival(class2, Exp(0))    # No direct arrivals    source.set_arrival(class3, Exp(0))    # No direct arrivals        # Service rates - same server, different service requirements per stage    queue1.set_service(class1, Exp(4))    # First visit: fast service    queue1.set_service(class2, Exp(0))    # No service for class2 at queue1    queue1.set_service(class3, Exp(2))    # Reentrant visit: slower service        queue2.set_service(class1, Exp(0))    # No service for class1 at queue2    queue2.set_service(class2, Exp(3))    # Service for class2    queue2.set_service(class3, Exp(0))    # No service for class3 at queue2        # Block 3: reentrant routing    P = model.init_routing_matrix()        # Stage 1: Source -> Queue1 (as class1) -> Queue2 (becomes class2)    P.add_route(class1, source, queue1, 1.0)    P.add_route(class1, queue1, queue2, 1.0)  # Class switching happens here        # Stage 2: Queue2 (as class2) -> Queue1 (becomes class3, reentrant)    P.add_route(class2, queue2, queue1, 1.0)  # Class switching happens here        # Stage 3: Queue1 (as class3) -> Sink (exit)    P.add_route(class3, queue1, sink, 1.0)        model.link(P)        return model# Create the modelmodel = gallery_mm1_reentrant()print(f"\nReentrant pattern: Queue1 -> Queue2 -> Queue1 (reentry) -> Exit")

## Reentrant Network Analysis

In this reentrant network:
- **Queue1** serves jobs twice (stages 1 and 3)
- **Queue2** serves jobs once (stage 2)
- **Load multiplication**: Queue1 handles 2x the external arrival rate
- **Bottleneck effects**: Queue1 may become heavily loaded

Total load on Queue1 = λ (stage 1) + λ (stage 3) = 2λ
Total load on Queue2 = λ (stage 2)

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
try:
    solver_mva = MVA(model)
    avg_table_mva = solver_mva.avg_table()
    print("\nMVA Solver:")
    print(avg_table_mva)
except Exception as e:
    print(f"MVA Solver error: {e}")

# CTMC Solver
try:
    solver_ctmc = CTMC(model, cutoff=10)
    avg_table_ctmc = solver_ctmc.avg_table()
    print("\nCTMC Solver:")
    print(avg_table_ctmc)
except Exception as e:
    print(f"CTMC Solver error: {e}")

# Fluid Solver
try:
    solver_fluid = FLD(model)
    avg_table_fluid = solver_fluid.avg_table()
    print("\nFluid Solver:")
    print(avg_table_fluid)
except Exception as e:
    print(f"Fluid Solver error: {e}")

In [ ]:
# Alternative simpler reentrant model using class switching
print("\n=== Alternative Reentrant Model (Class Switching) ===")

def create_reentrant_with_class_switch():
    """Create reentrant network using explicit class switching"""
    model_cs = Network('Reentrant-ClassSwitch')
    
    # Nodes
    source = Source(model_cs, 'Source')
    queue1 = Queue(model_cs, 'Queue1', SchedStrategy.FCFS)
    queue2 = Queue(model_cs, 'Queue2', SchedStrategy.FCFS)
    sink = Sink(model_cs, 'Sink')
    
    # Single class with class switching routing
    oclass = OpenClass(model_cs, 'Jobs')
    
    source.set_arrival(oclass, Exp(0.5))
    queue1.set_service(oclass, Exp(3))  # Average of both visits
    queue2.set_service(oclass, Exp(3))
    
    # Routing with feedback to create reentrant behavior
    P = model_cs.init_routing_matrix()
    P.add_route(oclass, source, queue1, 1.0)
    P.add_route(oclass, queue1, queue2, 0.8)  # Most go to queue2
    P.add_route(oclass, queue1, sink, 0.2)    # Some exit (reentrant jobs)
    P.add_route(oclass, queue2, queue1, 1.0)  # All return to queue1 (reentry)
    
    model_cs.link(P)
    return model_cs

model_cs = create_reentrant_with_class_switch()
solver_cs = MVA(model_cs)
avg_table_cs = solver_cs.avg_table()

print("Class Switching Reentrant Network:")
print(avg_table_cs)

# Analyze effective load multiplication
queue1_util = float(avg_table_cs.iloc[1, 1])  # Queue1 utilization
queue2_util = float(avg_table_cs.iloc[2, 1])  # Queue2 utilization

print(f"\nLoad Analysis:")
print(f"Queue1 utilization: {queue1_util:.3f} (handles multiple visits)")
print(f"Queue2 utilization: {queue2_util:.3f} (handles single visits)")
print(f"Load ratio Q1/Q2: {queue1_util/queue2_util:.2f}" if queue2_util > 0 else "Queue2 has no load")

In [ ]:
# Compare with equivalent tandem network (no reentrancy)
print("\n=== Comparison with Non-Reentrant Tandem ===")

def create_equivalent_tandem():
    """Create equivalent tandem network for comparison"""
    model_tandem = Network('Equivalent-Tandem')
    
    source = Source(model_tandem, 'Source')
    queue1 = Queue(model_tandem, 'Queue1', SchedStrategy.FCFS)
    queue2 = Queue(model_tandem, 'Queue2', SchedStrategy.FCFS)
    sink = Sink(model_tandem, 'Sink')
    
    oclass = OpenClass(model_tandem, 'Jobs')
    source.set_arrival(oclass, Exp(0.5))
    queue1.set_service(oclass, Exp(3))  # Same service rate
    queue2.set_service(oclass, Exp(3))  # Same service rate
    
    # Simple tandem routing
    P = model_tandem.init_routing_matrix()
    P.add_route(oclass, source, queue1, 1.0)
    P.add_route(oclass, queue1, queue2, 1.0)
    P.add_route(oclass, queue2, sink, 1.0)
    
    model_tandem.link(P)
    return model_tandem

model_tandem = create_equivalent_tandem()
solver_tandem = MVA(model_tandem)
avg_table_tandem = solver_tandem.avg_table()

print("Tandem Network (no reentrancy):")
print(avg_table_tandem)

# Compare performance
tandem_q1_util = float(avg_table_tandem.iloc[1, 1])
tandem_q2_util = float(avg_table_tandem.iloc[2, 1])
tandem_total_resp = float(avg_table_tandem.iloc[1, 2]) + float(avg_table_tandem.iloc[2, 2])

cs_total_resp = float(avg_table_cs.iloc[1, 2]) + float(avg_table_cs.iloc[2, 2])

print(f"\nPerformance Comparison:")
print(f"Tandem Network:")
print(f"  Q1 utilization: {tandem_q1_util:.3f}")
print(f"  Q2 utilization: {tandem_q2_util:.3f}")
print(f"  Total response: {tandem_total_resp:.3f}")

print(f"\nReentrant Network:")
print(f"  Q1 utilization: {queue1_util:.3f} (higher due to reentrancy)")
print(f"  Q2 utilization: {queue2_util:.3f}")
print(f"  Total response: {cs_total_resp:.3f}")

print(f"\nReentrancy Impact: {(cs_total_resp / tandem_total_resp):.2f}x response time increase")
print("Note: Reentrancy creates additional load and contention at revisited stations.")